In [95]:
#Fav / Dog Win percentage + odds filter
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
stake=100
datefilter='2024-01-01'

devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()

def hist_data_dog(surface, sex, odds_low,odds_high,date):
    data=pd.read_sql_query(f"Select distinct * from results_{surface}_1 where date > '{date}' ",con=devengine)#.to_clipboard(index=False)
    data["Fav_Odds"] = data.Fav_Odds.astype(float)
    data["Dog_Odds"] = data.Dog_Odds.astype(float)
    data['Fav_Win'] = data['Winner']==data['Fav']
    data['DogWin'] = data['Winner']!=data['Fav']
    data=data[(data['Fav_Odds']>=odds_low)&(data['Fav_Odds']<=odds_high)]
    data=data[data['Sex']==sex]
    return data
hard_womens=hist_data_dog('HArd','Womens',1,2,datefilter)
hard_mens=hist_data_dog('HArd','Mens',1,2,datefilter)
temp=pd.concat([hard_womens,hard_mens]).sort_values(by='Date')
temp = temp.reset_index(drop=True)
temp['Fav_Loss'] = (temp['Fav_Win'] == 0).astype(int)
temp['Fav_Loss_Count'] = temp.groupby('Fav')['Fav_Loss'].cumsum()
temp['Dog_Win'] = (temp['Fav_Win'] == 0).astype(int)
temp['Dog_Win_Count'] = temp.groupby('Dog')['Dog_Win'].cumsum()

In [111]:
#Female History
stake=100
date_filter=temp[temp['Date']>'2024-06-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.6].copy()
#oddsfilter=date_filter[date_filter['Dog_Odds']>=2.1]
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>1]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>1]
#oddsfilter=oddsfilter[oddsfilter['Elo_Fav']==oddsfilter['Dog']]
#oddsfilter=oddsfilter[oddsfilter['Dog_Top100']>.5]
#oddsfilter=oddsfilter[oddsfilter['Fav_Top100']<.5]
#oddsfilter=oddsfilter[oddsfilter['dog_rank']<200]
#oddsfilter=oddsfilter[oddsfilter['fav_rank']<300]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Womens']
len(oddsfilter[oddsfilter['Fav_Loss']==1])/len(oddsfilter),len(oddsfilter)
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter.to_csv('_HardResults_Womens.csv')

Total Profit: 7720.0 340 0.5882352941176471


In [112]:
#Male History
stake=100
date_filter=temp[temp['Date']>'2024-06-01']
oddsfilter=date_filter[date_filter['Fav_Odds']>1.6].copy()
#oddsfilter=oddsfilter[oddsfilter['Dog_Odds']<2]
#oddsfilter=oddsfilter[oddsfilter['Fav_Odds']<1.8].copy()
oddsfilter=oddsfilter[oddsfilter['Dog_Win_Count']>1]
oddsfilter=oddsfilter[oddsfilter['Fav_Loss_Count']>1]
#oddsfilter=oddsfilter[oddsfilter['Dog_Top100']>.5]
#oddsfilter=oddsfilter[oddsfilter['Fav_Top100']<.5]
#oddsfilter=oddsfilter[oddsfilter['dog_rank']>200]
#oddsfilter=oddsfilter[oddsfilter['fav_rank']>100]
oddsfilter=oddsfilter[oddsfilter['Sex']=='Mens']
len(oddsfilter[oddsfilter['Fav_Loss']==1])/len(oddsfilter),len(oddsfilter)
oddsfilter['Profit'] = oddsfilter.apply(
    lambda row: (row['Dog_Odds'] - 1) * stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = oddsfilter['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(oddsfilter),len(oddsfilter[oddsfilter['Dog']==oddsfilter['Winner']])/len(oddsfilter))
oddsfilter.to_csv('_HardResults_Mens.csv')

Total Profit: 11474.0 932 0.5429184549356223


In [146]:
hardmen=pd.read_csv(r'.\_HardResults_Mens.csv')
hardwommen=pd.read_csv(r'.\_HardResults_Womens.csv')
claymen=pd.read_csv(r'.\_ClayResults_Mens.csv')
claywommen=pd.read_csv(r'.\_ClayResults_Womens.csv')

combined=pd.concat([claymen,claywommen])

import pandas as pd
oddsfilter=combined.copy()

# Ensure date is datetime and sorted
oddsfilter['Date'] = pd.to_datetime(oddsfilter['Date'])
oddsfilter = oddsfilter.sort_values('Date').reset_index(drop=True)

# Initialize bankroll tracking
initial_bankroll = 260
daily_bankrolls = {}
current_bankroll = initial_bankroll

stakes = []
profits = []
bankrolls = []

# Group by date
for date, day_df in oddsfilter.groupby('Date'):
    stake = round(0.04 * current_bankroll)  # stake same for all matches on this day

    day_profits = []

    # Compute profit for each row in the day
    for _, row in day_df.iterrows():
        if row['DogWin'] == 1:
            profit = row['Dog_Odds'] * stake - stake
        else:
            profit = -stake
        day_profits.append(profit)

        # Append values for each row
        stakes.append(stake)
        profits.append(profit)
        bankrolls.append(current_bankroll)

    # Update bankroll after all bets for the day
    current_bankroll += sum(day_profits)

# Assign columns
oddsfilter['Stake'] = stakes
oddsfilter['Profit'] = profits
oddsfilter['Bankroll'] = bankrolls
oddsfilter.to_clipboard()

In [ ]:
today=pd.read_pickle('.\Hard_Today')
today=today[today['Fav_Odds']>1.6]
today=today[today['Dog_Win_Count']>0]
today=today[today['Fav_Loss_Count']>0]
today[today['Sex']=='Womens']

<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9572\3636087751.py:1: SyntaxWarning: invalid escape sequence '\H'
  today=pd.read_pickle('.\Hard_Today')


,Fav_Top100,Dog_Top100,Sex,Resulted,Time,Fav,fav_rank,fav_rank_high,dog_rank,dog_rank_high,...,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count


In [54]:
lowodds=pd.read_pickle(r'.\Hard_Today')
lowodds=lowodds[lowodds['Fav_Odds']<1.3].copy()
lowodds=lowodds[lowodds['Fav_Odds']>1]
favs=pd.read_csv('Fav_Hard.csv')
fav_list=favs['Name'].tolist()
lowodds=lowodds[(lowodds['Fav']).isin(fav_list)]
lowodds=lowodds[lowodds['dog_rank']>50]
lowodds

,Fav_Top100,Dog_Top100,Sex,Resulted,Time,Fav,fav_rank,fav_rank_high,dog_rank,dog_rank_high,...,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count


In [ ]:
today=pd.read_pickle('.\Hard_Today')
today=today[today['Fav_Odds']>1.6]
today=today[today['Fav_Loss_Count']>0]
today=today[today['Dog_Win_Count']>0]
#today=today[today['Elo_Fav']==today['Dog']]
#today=today[today['fav_rank']>100]
#today=today[today['dog_rank']>200]
today[today['Sex']!='Womens']

<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9572\2673991922.py:1: SyntaxWarning: invalid escape sequence '\H'
  today=pd.read_pickle('.\Hard_Today')


,Fav_Top100,Dog_Top100,Sex,Resulted,Time,Fav,fav_rank,fav_rank_high,dog_rank,dog_rank_high,...,Fav_Return%,fav_last_five_win_perc,Dog,Dog_Odds,dog_percent,Dog_Serve%,Dog_Return%,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count


In [ ]:
# Current date and time
current_datetime = datetime.datetime.now()#+ datetime.timedelta(days=-1)
# Convert to string
date_string = current_datetime.strftime("%Y-%m-%d")

today=pd.read_pickle('.\Hard_Today')
today=today[today['Fav_Odds']>1.6]
today=today[today['Dog_Win_Count']>0]
today=today[today['Fav_Loss_Count']>0]
today['Date']=date_string
today.to_sql('Preditions_Hard',con=devengine,if_exists='append')

<>:6: SyntaxWarning: invalid escape sequence '\H'
<>:6: SyntaxWarning: invalid escape sequence '\H'
C:\Users\Administrator\AppData\Local\Temp\ipykernel_9572\3208968739.py:6: SyntaxWarning: invalid escape sequence '\H'
  today=pd.read_pickle('.\Hard_Today')


0

In [76]:
predictions=pd.read_sql_table('Preditions_Hard',con=devengine)
results=temp
merged_df = pd.merge(
    predictions, 
    results[['Date', 'Fav', 'Dog', 'Winner']],  # Select only the Winner column from results
    on=['Date', 'Fav', 'Dog'], 
    how='left'  # Use left join to retain all rows from predictions
)
#merged_df_filtered=merged_df[merged_df['Dog_Win_Count']>2].copy()
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Loss_Count']>2]
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Odds']>1.7]

merged_df_filtered=merged_df[merged_df['Winner'].notna()].copy()
merged_df_filtered=merged_df_filtered[merged_df_filtered['Fav_Odds']>=1.6]
#merged_df_filtered=merged_df_filtered[merged_df_filtered['Date']=='2025-02-20']
merged_df_filtered['Profit'] = merged_df_filtered.apply(
    lambda row: (row['Dog_Odds'] * stake)-stake if row['Winner'] == row['Dog'] else -stake, axis=1
)

# Calculate total profit
total_profit = merged_df_filtered['Profit'].sum()

# Print the total profit
print("Total Profit:", total_profit, len(merged_df_filtered),len(merged_df_filtered[merged_df_filtered['Dog']==merged_df_filtered['Winner']])/len(merged_df_filtered))
merged_df_filtered


Total Profit: -143.0 49 0.4897959183673469


,index,Time,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,...,Fav_Return%,Dog_Serve%,Dog_Return%,fav_last_five_win_perc,dog_last_five_win_perc,Fav_Loss_Count,Dog_Win_Count,Date,Winner,Profit
0,4,01:05,1.86,1.89,Terence Atmane,Alexander Blockx,41%,27,Alexander Blockx,63%,...,0.00,NaN,NaN,0.2,0.2,15.0,11.0,2025-03-19,Alexander Blockx,89.0
1,5,01:05,1.82,1.98,Alycia Parks,Alycia Parks,38%,29,Varvara Gracheva,28%,...,0.24,0.54,0.30,0.6,0.4,8.0,9.0,2025-03-19,Alycia Parks,-100.0
2,19,05:20,1.71,2.13,Claire Liu,Claire Liu,100%,2,Linda Fruhvirtova,67%,...,0.37,0.66,0.35,0.8,0.4,8.0,7.0,2025-03-19,Linda Fruhvirtova,113.0
3,20,05:45,1.87,1.92,Gabriela Ruse Elena,Gabriela Ruse Elena,33%,9,Polina Kudermetova,50%,...,NaN,0.00,0.00,0.6,0.4,5.0,11.0,2025-03-19,Gabriela Ruse Elena,-100.0
4,23,08:10,1.83,1.97,Olga Danilovic,Hailey Baptiste,56%,9,Hailey Baptiste,42%,...,0.31,0.64,0.29,0.4,0.6,6.0,12.0,2025-03-19,Hailey Baptiste,97.0
5,4,01:10,1.88,1.92,Veronika Kudermetova,Veronika Kudermetova,47%,43,Xin Yu Wang,48%,...,0.31,NaN,NaN,0.6,0.4,18.0,7.0,2025-03-20,Veronika Kudermetova,-100.0
6,8,03:00,1.85,1.93,Ajla Tomljanovic,Ajla Tomljanovic,53%,15,Bernarda Pera,47%,...,0.38,0.73,0.40,0.8,0.4,5.0,12.0,2025-03-20,Bernarda Pera,93.0
7,17,05:55,1.77,2.04,Peyton Stearns,Katie Boulter,40%,42,Katie Boulter,50%,...,0.24,0.69,0.30,0.4,0.4,11.0,9.0,2025-03-20,Peyton Stearns,-100.0
8,1,01:05,1.78,2.02,Nuno Borges,Nuno Borges,49%,47,Zizou Bergs,44%,...,0.13,0.83,0.17,0.6,0.4,10.0,9.0,2025-03-21,Zizou Bergs,102.0
9,2,01:10,1.87,1.91,Gabriela Ruse Elena,Gabriela Ruse Elena,38%,8,Magdalena Frech,62%,...,NaN,0.61,0.30,0.8,0.2,5.0,15.0,2025-03-21,Gabriela Ruse Elena,-100.0
